In [ ]:
fig, ax = plt.subplots(figsize = (16,10))
ax.scatter(sqft_living_outliers_df.long, sqft_living_outliers_df.lat, zorder=1, alpha= 1, c=sqft_living_outliers_df.price, s=10)
ax.set_xlim(BBox[0],BBox[1])
ax.set_ylim(BBox[2],BBox[3])
ax.set_title('Plotting Data on Map')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.plot(-122.2,47.5983,c='r',markersize=5, marker='o') # Most expensive house
ax.imshow(map_img, zorder=0, extent = BBox, aspect= 'equal');

In [ ]:
#import chart_studio.plotly as py
import plotly.offline as py
import plotly.graph_objects as go

mapbox_access_token = 'pk.eyJ1IjoidmFkaW1icyIsImEiOiJja240c3JqaTExaXh4Mm5vMmx3NDNwOHUwIn0.tHSRuXCZ3UQxUnwNQRyZCQ'

#df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/Nuclear%20Waste%20Sites%20on%20American%20Campuses.csv')
site_lat = cropped_df.long
site_lon = cropped_df.lat
locations_name = cropped_df.price

data = [
    go.Scattermapbox(
        lat=list(site_lat),
        lon=list(site_lon),
        mode='markers',
        marker=dict(
            size=17,
            color='rgb(255, 0, 0)',
            opacity=0.7
        ),
        text=locations_name,
        hoverinfo='text'
    ),
    go.Scattermapbox(
        lat=site_lat,
        lon=site_lon,
        mode='markers',
        marker=dict(
            size=8,
            color='rgb(242, 177, 172)',
            opacity=0.7
        ),
        hoverinfo='none'
    )
]


layout = go.Layout(
    title='Nuclear Waste Sites on Campus',
    autosize=True,
    hovermode='closest',
    showlegend=False,
    mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=dict(
            lat=47.6,
            lon=-122
        ),
        pitch=0,
        zoom=9,
        style='light'
    ),
)

fig = dict(data=data, layout=layout)

#py.iplot(fig, filename='jupyter-Nuclear Waste Sites on American Campuses')
py(fig)
#fig.layout.update()
#fig.show()
#plot( fig, filename = 'candlestick-test-3.html', validate = False )